In [35]:
import os
import pandas as pd
import numpy as np
from scipy import stats as sps
from scipy.interpolate import interp1d
import re
from imp import reload

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)

In [2]:
url = 'https://covidtracking.com/api/v1/states/daily.csv'
states = pd.read_csv(url,
                     usecols=['date', 'state', 'positive'],
                     parse_dates=['date'],
                     index_col=['state', 'date'],
                     squeeze=True).sort_index()

In [72]:
raw_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
#drop NA fips
county_data = raw_data.dropna(subset=['FIPS'], axis='rows')
#drop counties that are unassigned/contain "Out of" or are NaN
county_data = county_data[np.invert(county_data.Admin2.str.contains("Unassigned|Out of").replace(np.nan, True))]

#get just FIPS and timeseries data of cases
county_data = county_data[np.append(["FIPS"],list(raw_data.columns[[re.match(r"[0-9]*/[0-9]*/[0-9]*", col)!=None for col in raw_data.columns]]))]
#make the name match the one used in CHAD for simplicity
county_data.rename(columns = {"FIPS": "CountyFIPS"}, inplace=True)
#set FIPS to be the index
county_data = county_data.set_index('CountyFIPS')
#convert columns to date format
county_data.columns = pd.DatetimeIndex(county_data.columns)
county_data.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,...,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23
CountyFIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,7,8,10,12,12,12,12,12,12,15,17,19,19,19,23,24,26,26,25,26,28,30,32,33
1003.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,18,19,20,24,28,29,29,38,42,44,56,59,66,71,72,87,91,101,103,109,112,117,123,132,143
1005.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,2,2,3,3,4,9,9,10,10,11,12,14,15,18,20,22,28,29,30
1007.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,3,3,4,4,4,5,7,8,9,9,11,13,16,17,17,18,22,24,26,28,32,32,34,33
1009.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,6,9,10,10,10,10,10,11,12,12,13,14,16,17,18,20,20,21,22,26,29,31


In [73]:
raw_data[raw_data.FIPS.isin(neg_counties)]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20
770,84018137.0,US,USA,840,18137.0,Ripley,Indiana,US,39.102356,-85.262127,"Ripley, Indiana, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,25,31,33,39,47,51,60,70,76,76,77,79,81,84,86,89,88,89,77,80,82,82,83,86,87
1086,84021177.0,US,USA,840,21177.0,Muhlenberg,Kentucky,US,37.214258,-87.146321,"Muhlenberg, Kentucky, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7,10,10,10,10,0,0,0,27,27,26,26,26,28,28,45,47,55,54,58,75,76,76,74,72
1859,84036053.0,US,USA,840,36053.0,Madison,New York,US,42.916539,-75.672666,"Madison, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,42,49,62,68,77,78,89,92,92,92,102,102,102,105,110,110,110,110,117,117,117,117,117,121,106
1866,84036067.0,US,USA,840,36067.0,Onondaga,New York,US,43.004919,-76.199712,"Onondaga, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,228,249,277,301,322,349,364,377,397,422,422,456,456,494,520,537,563,579,598,598,637,646,646,667,600
1872,84036079.0,US,USA,840,36079.0,Putnam,New York,US,41.426301,-73.749655,"Putnam, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,167,167,207,216,252,283,314,345,402,427,456,487,494,507,507,634,634,634,732,732,778,778,807,831,615
1879,84036093.0,US,USA,840,36093.0,Schenectady,New York,US,42.816688,-74.052783,"Schenectady, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,80,85,93,101,110,117,128,138,145,149,167,191,194,199,204,235,237,245,295,295,295,295,336,352,288
1883,84036101.0,US,USA,840,36101.0,Steuben,New York,US,42.268914,-77.382992,"Steuben, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,19,19,38,43,46,60,73,75,75,87,92,117,117,147,147,147,151,154,154,181,181,195,195,214,171
1887,84036109.0,US,USA,840,36109.0,Tompkins,New York,US,42.449458,-76.472298,"Tompkins, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,70,73,76,80,87,93,95,102,103,105,107,107,108,113,113,113,118,118,119,119,119,119,123,123,119
1888,84036111.0,US,USA,840,36111.0,Ulster,New York,US,41.890279,-74.262521,"Ulster, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,190,207,222,240,263,290,358,382,398,462,462,530,589,655,716,716,745,845,880,880,960,997,1018,1018,942
1895,84037001.0,US,USA,840,37001.0,Alamance,North Carolina,US,36.043470,-79.399761,"Alamance, North Carolina, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7,7,7,13,14,16,19,20,24,30,30,33,34,36,68,73,43,45,47,48,50,50,58,60,63


In [74]:
import realtime_rt as rt
reload(rt)

<module 'realtime_rt' from 'D:\\Documents\\work stuff\\COVID\\AFIT git\\Updated Rt\\realtime_rt.py'>

In [75]:
smoothed_data = county_data.apply(rt.prepare_cases, axis=1, return_original=False, cutoff=5)
smoothed_data = smoothed_data.dropna(how='all', axis='rows')
smoothed_data.min(axis='columns')[smoothed_data.min(axis='columns')<0]
# smoothed_data

,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,...,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23
CountyFIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,6.0,6.0,5.0,5.0,6.0,7.0,8.0,9.0
1017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,9.0,10.0,11.0,11.0,9.0,8.0,9.0,11.0,15.0,18.0,19.0,17.0,15.0,11.0,9.0,8.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0
1031.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0
1049.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
1055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,6.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,6.0,7.0
55127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,9.0,9.0,7.0,6.0,4.0,4.0,3.0,4.0
55133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,6.0,7.0,8.0,...,9.0,10.0,10.0,11.0,10.0,10.0,8.0,8.0,7.0,8.0,8.0,9.0,9.0,9.0,9.0,9.0,9.0,8.0,8.0,7.0,6.0,5.0,4.0,4.0,4.0


In [84]:
def get_posteriors(sr, r_t_range = np.linspace(0, 12, 12*100+1), sigma=0.25, track_log_likelihood = False):
    '''
    This calculates 
    
    Unfortunately there is not much better documentation to be provided here. 
    The math can by seen in https://github.com/k-sys/covid-19/blob/master/Realtime%20R0.ipynb.
    '''
    # Gamma is 1/serial interval
    # https://wwwnc.cdc.gov/eid/article/26/7/20-0282_article
    # https://www.nejm.org/doi/full/10.1056/NEJMoa2001316
    GAMMA = 1/7
    
    #drop na values
    sr = sr.dropna()
    
    # (1) Calculate Lambdas
    lam = sr[:-1].values * np.exp(GAMMA * (r_t_range[:, None] - 1))

    
    # (2) Calculate each day's likelihood
    likelihoods = pd.DataFrame(
        data = sps.poisson.pmf(sr[1:].values, lam),
        index = r_t_range,
        columns = sr.index[1:])
    
    # (3) Create the Gaussian Matrix
    process_matrix = sps.norm(loc=r_t_range,
                              scale=sigma
                             ).pdf(r_t_range[:, None]) 

    # (3a) Normalize all rows to sum to 1
    process_matrix /= process_matrix.sum(axis=0)
    
    # (4) Calculate the initial prior
    #prior0 = sps.gamma(a=4).pdf(r_t_range)
    prior0 = np.ones_like(r_t_range)/len(r_t_range) #uniform distribution
    prior0 /= prior0.sum()

    # Create a DataFrame that will hold our posteriors for each day
    # Insert our prior as the first posterior.
    posteriors = pd.DataFrame(
        index=r_t_range,
        columns=sr.index,
        data={sr.index[0]: prior0}
    )
    
    if track_log_likelihood:
        # We said we'd keep track of the sum of the log of the probability
        # of the data for maximum likelihood calculation.
        log_likelihood = 0.0

    # (5) Iteratively apply Bayes' rule
    for previous_day, current_day in zip(sr.index[:-1], sr.index[1:]):

        #(5a) Calculate the new prior (@ here is matrix multiplication) I'm still not sure about this step
        current_prior = process_matrix @ posteriors[previous_day]
        
        #(5b) Calculate the numerator of Bayes' Rule: P(k|R_t)P(R_t)
        numerator = likelihoods[current_day] * current_prior
        
        #(5c) Calcluate the denominator of Bayes' Rule P(k)
        denominator = np.sum(numerator)
        
        # Execute full Bayes' Rule
        posteriors[current_day] = numerator/denominator
        
        if track_log_likelihood:
            # Add to the running sum of log likelihoods
            log_likelihood += np.log(denominator)
    
    if track_log_likelihood:
        return posteriors, log_likelihood
    else:
        return posteriors

In [86]:
post = get_posteriors(smoothed_data.iloc[0])

In [87]:
process_matrix

array([[1.59576912, 1.59449302, 1.59067082, ..., 0.        , 0.        ,
        0.        ],
       [1.59449302, 1.59576912, 1.59449302, ..., 0.        , 0.        ,
        0.        ],
       [1.59067082, 1.59449302, 1.59576912, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.59576912, 1.59449302,
        1.59067082],
       [0.        , 0.        , 0.        , ..., 1.59449302, 1.59576912,
        1.59449302],
       [0.        , 0.        , 0.        , ..., 1.59067082, 1.59449302,
        1.59576912]])

In [90]:
post.iloc[:,-1]

0.00     1.455104e-03
0.01     1.507850e-03
0.02     1.560996e-03
0.03     1.614474e-03
0.04     1.668218e-03
             ...     
11.96    1.332270e-47
11.97    1.067019e-47
11.98    8.543215e-48
11.99    6.838171e-48
12.00    5.471771e-48
Name: 2020-04-23 00:00:00, Length: 1201, dtype: float64